# `core`
> set of functions and classes used across this package and usable for other packages

In [ ]:
#|default_exp core

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import Any, List, Optional

import configparser
import numpy as np
import sys

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# Validation tools

In [ ]:
#| export
def is_type(
    obj:Any,                 # object whose type to validate
    obj_type:type,                # expected type for `obj`
    raise_error:bool=False,  # when True, raise a ValueError is `obj` is not of the right type
)-> bool:                    # True when `obj` is of the right type, False otherwise 
    """Validate that `obj` is of type `obj_type`. Raise error in the negative when `raise_error` is `True`"""
    if not isinstance(obj_type, type): raise ValueError(f"{obj_type} is not a type")
    if isinstance(obj, obj_type): return True
    else:
        if raise_error: raise ValueError(f"passed object is not of type {obj_type}")
        else: return False

In [ ]:
show_doc(is_type)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L16){target="_blank" style="float:right; font-size:smaller"}

### is_type

>      is_type (obj:Any, obj_type:type, raise_error:bool=False)

Validate that `obj` is of type `obj_type`. Raise error in the negative when `raise_error` is `True`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| obj | Any |  | object whose type to validate |
| obj_type | type |  | expected type for `obj` |
| raise_error | bool | False | when True, raise a ValueError is `obj` is not of the right type |
| **Returns** | **bool** |  | **True when `obj` is of the right type, False otherwise** |

In [ ]:
is_type(obj='this is a string', obj_type=str)

True

In [ ]:
is_type(obj=np.ones(shape=(2,2)), obj_type=np.ndarray)

True

In [ ]:
#| export
def validate_path(
    path:str|Path,           # path to validate
    path_type:str='file',    # type of the target path: `'file'`, `'dir'` or `'any'`
    raise_error:bool=False,  # when True, raise a ValueError is path does not a file
)-> bool:                    # True when path is a valid path, False otherwise 
    """Validate that path is a Path or str and points to a real file or directory"""
    if isinstance(path, str): 
        path = Path(path)
    if (path_type=='file' and path.is_file()) or (path_type=='dir' and path.is_dir()) :
        return True
    if path_type=='any' and path.exists():
        return True
    else:
        if raise_error: raise ValueError(f"No file at {path.absolute()}. Check the path")
        else: return False

In [ ]:
show_doc(validate_path)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L29){target="_blank" style="float:right; font-size:smaller"}

### validate_path

>      validate_path (path:str|pathlib.Path, path_type:str='file',
>                     raise_error:bool=False)

Validate that path is a Path or str and points to a real file or directory

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | str \| Path |  | path to validate |
| path_type | str | file | type of the target path: `'file'`, `'dir'` or `'any'` |
| raise_error | bool | False | when True, raise a ValueError is path does not a file |
| **Returns** | **bool** |  | **True when path is a valid path, False otherwise** |

In [ ]:
path_file = Path('../data/img/IMG_001_512px.jpg')
validate_path(path_file)

True

In [ ]:
validate_path(path_file, path_type='any')

True

In [ ]:
path_dir = Path('../data')
validate_path(path_dir, path_type='dir')

True

In [ ]:
validate_path(path_dir, path_type='any')

True

In [ ]:
path_error = Path('../data/img/IIIMG_001_512px.jpg')
validate_path(path_error)

False

In [ ]:
#| export
def safe_path(
    path:str|Path, # path to validate
)-> Path:          # validated path as a  pathlib.Path
    """Return a Path object when given a valid path as a string or a Path, raise error otherwise"""
    validate_path(path, path_type='any', raise_error=True)
    if isinstance(path, str): 
        path = Path(path)
    return path

In [ ]:
show_doc(safe_path)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L46){target="_blank" style="float:right; font-size:smaller"}

### safe_path

>      safe_path (path:str|pathlib.Path)

Return a Path object when given a valid path as a string or a Path, raise error otherwise

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str \| Path | path to validate |
| **Returns** | **Path** | **validated path as a  pathlib.Path** |

# Access key files and directories

In [ ]:
#| export
def get_config_value(section:str,                        # section in the configparser cfg file
                     key:str,                            # key in the selected section
                     path_to_config_file:Path|str=None   # path to the cfg file
                    )-> Any :                            # the value corresponding to section>key>value 
    """Returns the value corresponding to the key-value pair in the configuration file (configparser format)"""
    # validate path_to_config_file
    if path_to_config_file is None:
        path_to_config_file = Path('/content/gdrive/MyDrive/private-across-accounts/config-api-keys.cfg')
    safe_path(path_to_config_file)

    configuration = configparser.ConfigParser()
    configuration.read(path_to_config_file)
    return configuration[section][key]

In [ ]:
show_doc(get_config_value)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L56){target="_blank" style="float:right; font-size:smaller"}

### get_config_value

>      get_config_value (section:str, key:str,
>                        path_to_config_file:pathlib.Path|str=None)

Returns the value corresponding to the key-value pair in the configuration file (configparser format)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| section | str |  | section in the configparser cfg file |
| key | str |  | key in the selected section |
| path_to_config_file | Path \| str | None | path to the cfg file |
| **Returns** | **Any** |  | **the value corresponding to section>key>value** |

By defaults (`path_to_config_file is None`), it is assumed that the configuration file is located in the `private-accross-accounts directory` on google drive. If not, a path to the file (`Path` or `str`) must be provided.

The configuration file is expected to be in the format used by the standard module `configparser` [documentation](https://docs.python.org/3/library/configparser.html)

```ascii
    [DEFAULT]
    key = value

    [section_name]
    key = value

    [section_name]
    key = value
```

In [ ]:
path2cfg = Path('../config-sample.cfg').resolve()
assert path2cfg.is_file(), f"{path2cfg} is not a file"

with open(path2cfg, 'r') as fp:
    print(fp.read())

[azure]
azure-api-key= dummy_api_key_for_azure

[kaggle]
kaggle_username = not_my_real_kaggle_name
kaggle_key = dummy_api_key_for_kaggle

[wandb]
api_key = dummy_api_key_for_wandb



In [ ]:
value = get_config_value(section='azure', key='azure-api-key', path_to_config_file=path2cfg)
assert value == 'dummy_api_key_for_azure'

In [ ]:
value = get_config_value(section='kaggle', key='kaggle_username', path_to_config_file=path2cfg)
assert value == 'not_my_real_kaggle_name'

In [ ]:
value = get_config_value(section='wandb', key='api_key', path_to_config_file=path2cfg)
assert value == 'dummy_api_key_for_wandb'

In [ ]:
#| export
class IsLocalMachine:
    """Callable singleton class to identify if current machine was registered as local machine or not"""
    
    _instance = None
    _config_dir = '.ecutilities'
    _config_fname = 'ecutilities.cfg'
    
    def __new__(cls):
        if cls._instance is None:
            cls.home = Path.home().absolute()
            cls._instance = super().__new__(cls)
        return cls._instance
    
    @property
    def os(self): return sys.platform
    
    @property
    def home(self): return Path.hone().absolute()
    
    @property
    def p2config(self): return self.home / self._config_dir / self._config_fname
    
    def __call__(self): return self.is_local
    
    def read_config(self):
        """Read config from the configuration file if it exists and return an empty config in does not"""
        cfg = configparser.ConfigParser()
        if self.p2config.is_file(): 
            cfg.read(self.p2config)
        else:
            cfg.add_section('Infra')
        return cfg
        
    @property
    def is_local(self):
        """Return `True` if the current machine was registered as a local machine"""
        cfg = self.read_config()
        return cfg['Infra'].getboolean('registered_as_local', False)
    
    def register_as_local(self):
        """Update the configuration file to register the machine as local machine"""
        cfg = self.read_config()
        os.makedirs(self.home/self._config_dir, exist_ok=True)
        cfg['Infra']['registered_as_local'] = 'True'
        with open(self.p2config, 'w') as fp:
            cfg.write(fp)
        return cfg

In [ ]:
show_doc(IsLocalMachine)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L71){target="_blank" style="float:right; font-size:smaller"}

### IsLocalMachine

>      IsLocalMachine ()

Callable singleton class to identify if current machine was registered as local machine or not

Several options to check whether the machine was registered as local or not:

In [ ]:
islocal = IsLocalMachine()
islocal.is_local

True

In [ ]:
islocal()

True

In [ ]:
IsLocalMachine().is_local

True

In [ ]:
IsLocalMachine()()

True

In [ ]:
show_doc(IsLocalMachine.register_as_local)


---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L110){target="_blank" style="float:right; font-size:smaller"}

### IsLocalMachine.register_as_local

>      IsLocalMachine.register_as_local ()

Update the configuration file to register the machine as local machine

Use this method to register the current machine as local machine. Only needs to be used once on a machine. Do not use on cloud VMs

> **Technical Note**:
>
> The configuration file is located at a standard location, which varies depending on the OS:
> 
> - Windows:
>    - home is `C:\Users\username`
>    - application data in `C:\Users\username\AppData/Local/...` or `C:\Users\username\AppData\Roaming\...` (see [StackExchange](https://superuser.com/questions/21458/why-are-there-directories-called-local-locallow-and-roaming-under-users-user))
>    - application also can be loaded under a dedicated directory under `C:\Users\username` like `C:\Users\username\.conda\...`
>
> - Linux:
>     - home is `/home/username`
>     - application data in a file or dedicated directory `/home/username/` s.a.:
>         - file in home directory, e.g. `.gitconfig`
>         - file in an application dedicated directory, e.g. `/home/username/.conda/...`
> 
> `ecutilities` places the configuration file in a dedicated directory in the home directory:
> - `C:\Users\username\.ecutilities\ecutilities.cfg`
> - `/home/username/.ecutilities/ecutilities.cfg`
> 
> 
> Retrieve the OS:
> ```python
> sys.platform
> ```
> ```shell
> win32           with Windows
> linux           with linux
> darwin          with macOs
> ```
> 
> Accessing the correct path depending on the OS:
> ```python
> Path().home().absolute()
> ```
> ```shell
> WindowsPath('C:/Users/username') with Windows
> Path('/home/username')           with linux
> ``` 
> 

# File structure exploration

In [ ]:
#| export
def files_in_tree(
    path: str|Path,               # path to the directory to scan  
    pattern: str|None = None      # pattern (glob style) to match in file name to filter the content
):
    """List files in directory and its subdiretories, print tree starting from parent directory"""
    validate_path(path, path_type='dir', raise_error=True)

    pattern = '*' if pattern is None else f"*{pattern}*"
    parents = [p.name for p in path.parents]
    paths = []
    pad = ' ' * 2
    idx = 0
    print(f"{parents[0]}")
    print(f"{pad}|--{path.name}")
    for f in [p for p in path.glob(pattern) if p.is_file()]:
        paths.append(f)
        print(f"{pad}|{pad*2}|--{f.name} ({idx})")
        idx += 1
    for d in [p for p in path.iterdir() if p.is_dir()]:
        print(f"{pad}|{pad*2}|--{d.name}")
        for f in [p for p in d.glob(pattern) if p.is_file()]:
            paths.append(f)
            print(f"{pad}|{pad*2}|{pad*2}|--{f.name} ({idx})")
            idx += 1
    return paths

In [ ]:
show_doc(files_in_tree)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L120){target="_blank" style="float:right; font-size:smaller"}

### files_in_tree

>      files_in_tree (path:str|pathlib.Path, pattern:str|None=None)

List files in directory and its subdiretories, print tree starting from parent directory

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | str \| Path |  | path to the directory to scan |
| pattern | str \| None | None | pattern (glob style) to match in file name to filter the content |

In [ ]:
p2dir = Path('').resolve()
print(p2dir, '\n')

files = files_in_tree(p2dir)
print(f"List of {len(files)} files when unfiltered")

/home/vtec/projects/ec-packages/ecutilities/nbs-dev 

ecutilities
  |--nbs-dev
  |    |--0_02_plotting.ipynb (0)
  |    |--2_01_image_utils.ipynb (1)
  |    |--1_01_eda_stats_utils.ipynb (2)
  |    |--0_01_ipython.ipynb (3)
  |    |--0_00_core.ipynb (4)
  |    |--.last_checked (5)
  |    |--sidebar.yml (6)
  |    |--1_02_ml.ipynb (7)
  |    |--index.ipynb (8)
  |    |--nbdev.yml (9)
  |    |--9_01_dev_utils.ipynb (10)
  |    |--styles.css (11)
  |    |--_quarto.yml (12)
  |    |--.ipynb_checkpoints
  |    |    |--0_02_plotting-checkpoint.ipynb (13)
  |    |    |--9_01_dev_utils-checkpoint.ipynb (14)
  |    |    |--0_01_ipython-checkpoint.ipynb (15)
  |    |    |--0_00_core-checkpoint.ipynb (16)
  |    |    |--1_01_eda_stats_utils-checkpoint.ipynb (17)
  |    |    |--index-checkpoint.ipynb (18)
  |    |    |--2_01_image_utils-checkpoint.ipynb (19)
  |    |    |--1_02_ml-checkpoint.ipynb (20)
List of 21 files when unfiltered


In [ ]:
files = files_in_tree(p2dir, pattern='ipynb')
print(f"List of {len(files)} files when filtered")

ecutilities
  |--nbs-dev
  |    |--0_02_plotting.ipynb (0)
  |    |--2_01_image_utils.ipynb (1)
  |    |--1_01_eda_stats_utils.ipynb (2)
  |    |--0_01_ipython.ipynb (3)
  |    |--0_00_core.ipynb (4)
  |    |--1_02_ml.ipynb (5)
  |    |--index.ipynb (6)
  |    |--9_01_dev_utils.ipynb (7)
  |    |--.ipynb_checkpoints
  |    |    |--0_02_plotting-checkpoint.ipynb (8)
  |    |    |--9_01_dev_utils-checkpoint.ipynb (9)
  |    |    |--0_01_ipython-checkpoint.ipynb (10)
  |    |    |--0_00_core-checkpoint.ipynb (11)
  |    |    |--1_01_eda_stats_utils-checkpoint.ipynb (12)
  |    |    |--index-checkpoint.ipynb (13)
  |    |    |--2_01_image_utils-checkpoint.ipynb (14)
  |    |    |--1_02_ml-checkpoint.ipynb (15)
List of 16 files when filtered


In [ ]:
#| export
def nbs_root_dir(
    path:str|Path|None = None, # path from where to seek for notebook parent directory
    pattern:str = 'nbs',       # pattern to identify the nbs directory
)-> Path:                      # path of the parent directory
    """Climb directory tree up to directory including pattern ('nbs'), and return its path"""
    if path is None: path = Path()
    path = safe_path(path).absolute()
    tree = [path.name] + [p.name for p in path.parents]
    mask = [True if n.startswith(pattern) else False for n in tree]
    tree = tree[mask.index(True):]
    tree.reverse()
    nbs = Path('/'.join(tree))
    return nbs

In [ ]:
show_doc(nbs_root_dir)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/core.py#L147){target="_blank" style="float:right; font-size:smaller"}

### nbs_root_dir

>      nbs_root_dir (path:str|pathlib.Path|None=None, pattern:str='nbs')

Climb directory tree up to directory including pattern ('nbs'), and return its path

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | str \| Path \| None | None | path from where to seek for notebook parent directory |
| pattern | str | nbs | pattern to identify the nbs directory |
| **Returns** | **Path** |  | **path of the parent directory** |

In [ ]:
nbs = nbs_root_dir()
nbs

Path('/home/vtec/projects/ec-packages/ecutilities/nbs-dev')

In [ ]:
nbs = nbs_root_dir(Path('../nbs/sandbox.ipynb').resolve())
nbs

Path('/home/vtec/projects/ec-packages/ecutilities/nbs')

In [ ]:
#| hide
nbdev_export()